In [1]:
"""
pip install opencv-python  # required for cv2
"""

import cv2
import numpy as np
from pathlib import Path
from glob import glob

In [2]:
def collate_images(background_path, image_paths, output_dir=Path("output"), scale_factor=0.8, prefix="", suffix="bgd"):
    """
    Collates images with a background, preserving aspect ratio and using a fixed scale factor.

    Args:
        background_path: Path to the background image.
        image_paths: List of paths to images to be collated.
        output_dir: Output directory Path for the collated images.
        scale_factor: The scaling factor for the images (0.0 - 1.0).
        suffix: added to the new output filename
    """
    # Ensure output directory exists
    output_dir.mkdir(parents=True, exist_ok=True)
    
    try:
        background = cv2.imread(str(background_path))
    except cv2.error as e:
        print(f"Error reading background image: {e}")
        return False

    bg_height, bg_width, _ = background.shape

    ok_images = 0
    for image_path in image_paths:
        print(f"Collating {image_path} ...")
        try:
            image = cv2.imread(str(image_path))
        except cv2.error as e:
            print(f"Error reading image: {image_path}, {e}")
            continue  # Skip to the next image            

        if image is None: 
            continue
            
        img_height, img_width, _ = image.shape

        try:
            # Determine the larger dimension and calculate new size based on scale factor
            if img_width > img_height:
                new_width = int(bg_width * scale_factor)
                new_height = int(img_height * (new_width / img_width))
            else:
                new_height = int(bg_height * scale_factor)
                new_width = int(img_width * (new_height / img_height))

            # Resize image
            resized_image = cv2.resize(image, (new_width, new_height))

            # Create a mask for the image
            mask = np.zeros_like(resized_image)
            mask[0:new_height, 0:new_width] = 255

            # Calculate position for placing the image on the background
            x = (bg_width - new_width) // 2
            y = (bg_height - new_height) // 2

            # Create a copy of the background image
            result = background.copy()

            # Put the resized image on the background
            result[y:y+new_height, x:x+new_width] = resized_image

            # Save the resulting image
            if prefix:
                new_filename = f"{prefix}__" + Path(image_path).stem + f"-{suffix}" + Path(image_path).suffix
            else:
                new_filename = Path(image_path).stem + f"-{suffix}" + Path(image_path).suffix
            output_path = output_dir / Path(new_filename)
            print(f"\tOutput file: {output_path}")
            cv2.imwrite(str(output_path), result)
            ok_images += 1
        except Exception as e:
            print(f"\t[ERROR] Failed processing '{image_path}'\n {e}")
            continue  # Skip to the next image    
            
    return len(image_paths), ok_images

In [3]:
background_path = "00-background-stary-universe.png"

In [4]:
ROOT_DIR = "1-A-Brilliant-Mind"
ITER_DIR = "iter-2"

In [5]:
source_images = glob(f"{ROOT_DIR}/original/*")
image_paths = [x for x in source_images if "-bgd" not in x]
total_images, ok_images = collate_images(Path(background_path), image_paths, output_dir=Path(f"{ROOT_DIR}/{ITER_DIR}"))
print(f"input_images, total_images, ok_images = {len(source_images)}, {total_images}, {ok_images}")

Collating 1-A-Brilliant-Mind/original\01-1-uewb_06_img0421.jpg ...
	Output file: 1-A-Brilliant-Mind\iter-2\01-1-uewb_06_img0421-bgd.jpg
Collating 1-A-Brilliant-Mind/original\01-2-fermi.png ...
	Output file: 1-A-Brilliant-Mind\iter-2\01-2-fermi-bgd.png
Collating 1-A-Brilliant-Mind/original\03-in-chicago.png ...
	Output file: 1-A-Brilliant-Mind\iter-2\03-in-chicago-bgd.png
Collating 1-A-Brilliant-Mind/original\04-1-td-lee-011.jpeg ...
	Output file: 1-A-Brilliant-Mind\iter-2\04-1-td-lee-011-bgd.jpeg
Collating 1-A-Brilliant-Mind/original\04-2-NPG-NPG_98_112.jpg ...
	Output file: 1-A-Brilliant-Mind\iter-2\04-2-NPG-NPG_98_112-bgd.jpg
Collating 1-A-Brilliant-Mind/original\05-T.D. Lee-IAS-edu-cropped.png ...
	Output file: 1-A-Brilliant-Mind\iter-2\05-T.D. Lee-IAS-edu-cropped-bgd.png
Collating 1-A-Brilliant-Mind/original\06-lee-yang-niels-bohr.jpeg ...
	Output file: 1-A-Brilliant-Mind\iter-2\06-lee-yang-niels-bohr-bgd.jpeg
Collating 1-A-Brilliant-Mind/original\07-1-td-lee-at-columbia-univ-cropp

### Compare iter-2 vs iter-1

In [20]:
iter1_images = glob(f"{ROOT_DIR}/iter-1/*")
iter2_images = glob(f"{ROOT_DIR}/iter-2/*")
len(iter1_images), len(iter2_images)

(40, 40)

In [21]:
set([x.replace("/iter-1", "") for x in iter1_images]).difference(set([x.replace("/iter-2", "") for x in iter2_images]))

set()

In [22]:
set([x.replace("/iter-2", "") for x in iter2_images]).difference(set([x.replace("/iter-1", "") for x in iter1_images]))

set()